![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/18.Text2SQL.ipynb)

# Text2SQL

In [2]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

license_keys.keys()


Saving workshop_license_keys_365.json to workshop_license_keys_365 (1).json


dict_keys(['PUBLIC_VERSION', 'JSL_VERSION', 'SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'SPARK_OCR_LICENSE', 'SPARK_OCR_SECRET'])

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys['SECRET']

os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
version = license_keys['PUBLIC_VERSION']
jsl_version = license_keys['JSL_VERSION']

! pip install --ignore-installed -q pyspark==2.4.4

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

! pip install --ignore-installed -q spark-nlp==$version

import sparknlp

print (sparknlp.version())

import json
import os
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

spark = sparknlp_jsl.start(secret)

# sparknlp_jsl.start(secret, public=version) if you want to start with different version of public sparknlp

## Convert schema json from SqLite schema

### explore SqLite tables

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/text2sql/university_basketball.sqlite

In [46]:
import sqlite3

conn = sqlite3.connect('university_basketball.sqlite')

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('basketball_match',), ('university',)]

In [47]:
import pandas as pd

from IPython.display import display, HTML

for table_name in tables:
    table_name = table_name[0]
    table = pd.read_sql_query("SELECT * from %s" % table_name, conn)
    display (table)

,Team_ID,School_ID,Team_Name,ACC_Regular_Season,ACC_Percent,ACC_Home,ACC_Road,All_Games,All_Games_Percent,All_Home,All_Road,All_Neutral
0,1,1,North Carolina,14–2,.875,6–2,8–0,35–2,0.946,14–2,13–0,9–1
1,2,2,Duke,13–3,.813,7–1,6–2,28–6,0.824,15–1,8–2,5–3
2,3,4,Clemson,10–6,.625,7–1,3–5,24–10,0.706,14–2,6–5,4–3
3,4,5,Virginia Tech,9–7,.563,6–2,3–5,21–14,0.600,14–3,4–8,3–3


,School_ID,School,Location,Founded,Affiliation,Enrollment,Nickname,Primary_conference
0,1,University of Delaware,"Newark, DE",1743.0,Public,19067.0,Fightin' Blue Hens,Colonial Athletic Association ( D-I )
1,2,Lebanon Valley College,"Annville, PA",1866.0,Private/Methodist,2100.0,Flying Dutchmen,MAC Commonwealth Conference ( D-III )
2,3,University of Rhode Island,"Kingston, RI",1892.0,Public,19095.0,Rams,Atlantic 10 Conference ( D-I )
3,4,Rutgers University,"New Brunswick, NJ",1766.0,Public,56868.0,Scarlet Knights,American Athletic Conference ( D-I )
4,5,Stony Brook University,"Stony Brook, NY",1957.0,Public,23997.0,Seawolves,America East Conference ( D-I )


### convert to tetx2SQL format

In [12]:
from sparknlp_jsl._tf_graph_builders.text2sql.util import sqlite2json

schema_json_path = 'schema_converted.json'

sqlite2json("university_basketball.sqlite",schema_json_path)


reading db:  university_basketball.sqlite


## Prepare DB schema

This is a one-time process for every new db schema that you want to work on

In [13]:
def prepare_db_schema(schema_json_path, output_json_path):

  document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  tables = Text2SQLSchemaParser() \
      .setOutputCol("table_metadata_chunk") \
      .setSchemaPath(schema_json_path) \
      .setInputCols(["document"])

  chunk2doc = Chunk2Doc() \
      .setInputCols(["table_metadata_chunk"]) \
      .setOutputCol("table_metadata_doc")

  table_tokenizer = Tokenizer() \
      .setOutputCol("table_token") \
      .setInputCols(["table_metadata_doc"])

  table_embedding = WordEmbeddingsModel.pretrained("glove_6B_300", "xx") \
      .setInputCols(["table_metadata_doc", "table_token"]) \
      .setOutputCol("table_embedding")

  table_chunk_embeddings = ChunkEmbeddings() \
      .setOutputCol("table_metadata_chunk_embedding") \
      .setInputCols("table_metadata_chunk", "table_embedding")

  table_exporter = Text2SQLSchemaExporter()\
      .setInputCols(["table_metadata_chunk_embedding","table_metadata_chunk"])\
      .setOutputPath(output_json_path)

  table_pl = Pipeline() \
      .setStages([
      document,
      tables,
      chunk2doc,
      table_tokenizer,
      table_embedding,
      table_chunk_embeddings,
      table_exporter
  ])

  data = spark.createDataFrame([
              [1, ""]]) \
              .toDF("id", "text").cache()

  table_model = table_pl.fit(data).transform(data)

  print (output_json_path, 'is created and saved')

  return table_model.show()

In [52]:
schema_json_path = 'schema_converted.json'
output_json_path = "db_embeddings.json"

prepare_db_schema(schema_json_path, output_json_path)

glove_6B_300 download started this may take some time.
Approximate size to download 426.2 MB
[OK!]
db_embeddings.json is created and saved
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------------+--------------------+
| id|text|            document|table_metadata_chunk|  table_metadata_doc|         table_token|     table_embedding|table_metadata_chunk_embedding|              export|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------------+--------------------+
|  1|    |[[document, 0, -1...|[[chunk, 0, 15, b...|[[document, 0, 15...|[[token, 0, 9, ba...|[[word_embeddings...|          [[word_embeddings...|[[chunk, 0, 15, b...|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------------+--------------------+



## Prepare Text2SQL pipeline

This is a one-time process for every new db schema that you want to work on

In [16]:
def get_text2sql_model (schema_json_path, output_json_path):

  question_document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  sentence_detector = SentenceDetectorDLModel.pretrained()\
      .setInputCols("document")\
      .setOutputCol("sentence")

  tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

  question_embbeding = WordEmbeddingsModel.pretrained("glove_6B_300", "xx") \
      .setInputCols(["sentence", "token"]) \
      .setOutputCol("question_embedding")

  text2sql_model = Text2SQLModel.pretrained('text2sql_glove', 'en', 'clinical/models') \
              .setSchemaPath(schema_json_path) \
              .setTableEmbeddingPath(output_json_path)\
              .setInputCols(["token", "question_embedding", "chunk_emb", "table_metadata_chunk"]) \
              .setOutputCol("sql")

  sql_pipeline = Pipeline(stages=[
              question_document,
              sentence_detector,
              tokenizer,
              question_embbeding,
              text2sql_model
          ])

  data = spark.createDataFrame([[""]]).toDF("text")

  sql_prediction_model = sql_pipeline.fit(data)

  sql_prediction_light = LightPipeline(sql_prediction_model)

  print ('text2sql prediction model is built')

  return sql_prediction_light

In [53]:
sql_prediction_light = get_text2sql_model (schema_json_path, output_json_path)

sentence_detector_dl download started this may take some time.
Approximate size to download 307.2 KB
[OK!]
glove_6B_300 download started this may take some time.
Approximate size to download 426.2 MB
[OK!]
text2sql_glove download started this may take some time.
Approximate size to download 37.6 MB
[OK!]
text2sql prediction model is built


## Example queries

In [57]:
import sqlparse

def annotate_and_print(question, sql_light=sql_prediction_light, markdown=False, param=None):

    sql = sql_light.annotate(question)["sql"][0]

    print(sqlparse.format(sql, reindent=True, keyword_case='upper'))
    print("\n")

    if markdown:
      print (pd.read_sql(sql,conn,params=param).to_markdown())
    else:
      display(pd.read_sql(sql,conn,params=param))
    

In [55]:
annotate_and_print("What are the enrollment and primary conference for the university which was founded the earliest?")

SELECT T1.Enrollment,
       T1.Primary_conference
FROM university AS T1
ORDER BY T1.Founded ASC
LIMIT 1




,Enrollment,Primary_conference
0,19067.0,Colonial Athletic Association ( D-I )


In [58]:
annotate_and_print("What are the enrollment and primary conference for the university which was founded the earliest?", markdown=True)

SELECT T1.Enrollment,
       T1.Primary_conference
FROM university AS T1
ORDER BY T1.Founded ASC
LIMIT 1


|    |   Enrollment | Primary_conference                    |
|---:|-------------:|:--------------------------------------|
|  0 |        19067 | Colonial Athletic Association ( D-I ) |


In [ ]:
annotate_and_print("What is the total and minimum enrollment of all schools?")

SELECT sum(T1.Enrollment),
       min(T1.Enrollment)
FROM university AS T1




,sum(T1.Enrollment),min(T1.Enrollment)
0,121127.0,2100.0


In [ ]:
annotate_and_print("Return the total and minimum enrollments across all schools.")

SELECT sum(T1.All_Neutral),
       min(T1.All_Neutral)
FROM basketball_match AS T1




,sum(T1.All_Neutral),min(T1.All_Neutral)
0,21.0,3–3


In [ ]:
annotate_and_print("Find the total student enrollment for different affiliation type schools.")

SELECT sum(T1.Enrollment),
       T1.Affiliation
FROM university AS T1
GROUP BY T1.Affiliation




,sum(T1.Enrollment),Affiliation
0,2100.0,Private/Methodist
1,119027.0,Public


In [ ]:
annotate_and_print("Find how many different affiliation types there are.")

SELECT count(T1.Affiliation)
FROM university AS T1




,count(T1.Affiliation)
0,5


## Use case: Hospital records

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/text2sql/hospital_records.sqlite

In [61]:
import sqlite3

conn = sqlite3.connect('/content/hospital_records.sqlite')

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('Physician',),
 ('Department',),
 ('Affiliated_With',),
 ('Procedures',),
 ('Trained_In',),
 ('Patient',),
 ('Nurse',),
 ('Appointment',),
 ('Medication',),
 ('Prescribes',),
 ('Block',),
 ('Room',),
 ('On_Call',),
 ('Stay',),
 ('Undergoes',)]

In [62]:
import pandas as pd

from IPython.display import display, HTML

for table_name in tables:
    table_name = table_name[0]
    table = pd.read_sql_query("SELECT * from %s" % table_name, conn)
    print (table_name)
    display (table.head(5))
    print ('========')

Physician


,EmployeeID,Name,Position,SSN
0,1,John Dorian,Staff Internist,111111111
1,2,Elliot Reid,Attending Physician,222222222
2,3,Christopher Turk,Surgical Attending Physician,333333333
3,4,Percival Cox,Senior Attending Physician,444444444
4,5,Bob Kelso,Head Chief of Medicine,555555555


Department


,DepartmentID,Name,Head
0,1,General Medicine,4
1,2,Surgery,7
2,3,Psychiatry,9


Affiliated_With


...

,Nurse,BlockFloor,BlockCode,OnCallStart,OnCallEnd
0,101,1,1,2008-11-04 11:00,2008-11-04 19:00
1,101,1,2,2008-11-04 11:00,2008-11-04 19:00
2,102,1,3,2008-11-04 11:00,2008-11-04 19:00
3,103,1,1,2008-11-04 19:00,2008-11-05 03:00
4,103,1,2,2008-11-04 19:00,2008-11-05 03:00


Stay


,StayID,Patient,Room,StayStart,StayEnd
0,3215,100000001,111,2008-05-01,2008-05-04
1,3216,100000003,123,2008-05-03,2008-05-14
2,3217,100000004,112,2008-05-02,2008-05-03


Undergoes


,Patient,Procedures,Stay,DateUndergoes,Physician,AssistingNurse
0,100000001,6,3215,2008-05-02,3,101
1,100000001,2,3215,2008-05-03,7,101
2,100000004,1,3217,2008-05-07,3,102
3,100000004,5,3217,2008-05-09,6,105
4,100000001,7,3217,2008-05-10,7,101


In [63]:
from sparknlp_jsl._tf_graph_builders.text2sql.util import sqlite2json

sqlite2json("hospital_records.sqlite","hospital_schema_converted.json")


reading db:  hospital_records.sqlite


In [64]:
schema_json_path = "hospital_schema_converted.json"

output_json_path = "hospital_db_embeddings.json"

prepare_db_schema(schema_json_path, output_json_path)

hospital_sql_prediction_light = get_text2sql_model (schema_json_path, output_json_path)

glove_6B_300 download started this may take some time.
Approximate size to download 426.2 MB
[OK!]
hospital_db_embeddings.json is created and saved
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------------+--------------------+
| id|text|            document|table_metadata_chunk|  table_metadata_doc|         table_token|     table_embedding|table_metadata_chunk_embedding|              export|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------------+--------------------+
|  1|    |[[document, 0, -1...|[[chunk, 0, 8, ph...|[[document, 0, 8,...|[[token, 0, 8, ph...|[[word_embeddings...|          [[word_embeddings...|[[chunk, 0, 8, ph...|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------------+--------------------+

sentence_de

In [65]:
annotate_and_print("Find the id of the appointment with the most recent start date", hospital_sql_prediction_light)

SELECT T1.Appointment
FROM Prescribes AS T1
JOIN Appointment AS T2 ON T1.Appointment = T2.AppointmentID
ORDER BY T2.Start DESC
LIMIT 1




,Appointment
0,86213939


In [66]:
annotate_and_print("What is the name of the patient who made the most recent appointment", hospital_sql_prediction_light, markdown=True)

SELECT T1.Name
FROM Patient AS T1
JOIN Appointment AS T2 ON T1.SSN = T2.Patient
ORDER BY T2.Start DESC
LIMIT 1


|    | Name       |
|---:|:-----------|
|  0 | Dennis Doe |


In [35]:
annotate_and_print("How many patients stay in room 112?", hospital_sql_prediction_light, ['112'])

SELECT count(T1.Patient)
FROM Stay AS T1
WHERE T1.Room = ?




,count(T1.Patient)
0,1


In [36]:
annotate_and_print("Find the patient who most recently stayed in room 111.",hospital_sql_prediction_light, ["111"])

SELECT T1.Patient
FROM Stay AS T1
JOIN Patient AS T3 ON T1.Patient = T3.SSN
JOIN Prescribes AS T2 ON T3.SSN = T2.Patient
WHERE T1.Room = ?
ORDER BY T2.Date DESC
LIMIT 1




,Patient
0,100000001


In [38]:
annotate_and_print("What is the name of the nurse has the most appointments?", hospital_sql_prediction_light)

SELECT T1.Name
FROM Nurse AS T1
JOIN Appointment AS T2 ON T1.EmployeeID = T2.PrepNurse
GROUP BY T2.prepnurse
ORDER BY count(*) DESC
LIMIT 1




,Name
0,Carla Espinosa


In [68]:
annotate_and_print("What is the name of the nurse has the most appointments?", hospital_sql_prediction_light, markdown=True)

SELECT T1.Name
FROM Nurse AS T1
JOIN Appointment AS T2 ON T1.EmployeeID = T2.PrepNurse
GROUP BY T2.prepnurse
ORDER BY count(*) DESC
LIMIT 1


|    | Name           |
|---:|:---------------|
|  0 | Carla Espinosa |


In [40]:
annotate_and_print("How many patients do each physician take care of? List their names and number of patients they take care of.", hospital_sql_prediction_light)

SELECT T1.Name,
       count(*)
FROM Physician AS T1
JOIN Patient AS T2 ON T1.EmployeeID = T2.PCP
GROUP BY T1.Name




,Name,count(*)
0,Christopher Turk,1
1,Elliot Reid,2
2,John Dorian,1


In [67]:
annotate_and_print("How many patients do each physician take care of? List their names and number of patients they take care of.", hospital_sql_prediction_light, markdown=True)

SELECT T1.Name,
       count(*)
FROM Physician AS T1
JOIN Patient AS T2 ON T1.EmployeeID = T2.PCP
GROUP BY T1.Name


|    | Name             |   count(*) |
|---:|:-----------------|-----------:|
|  0 | Christopher Turk |          1 |
|  1 | Elliot Reid      |          2 |
|  2 | John Dorian      |          1 |
